In [1]:
#Import Dependencies

import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
#Import NBA CSV Files

players_csv = "players.csv"
player_data_csv = "player_data.csv"
season_stats_csv = "Seasons_stats.csv"

df_players = pd.read_csv(players_csv)
df_player_data = pd.read_csv(player_data_csv)
df_season_stats = pd.read_csv(season_stats_csv)

In [3]:
#Read Header
df_player_data.head()

,name,year_start,year_end,position,height_feet,height_inches,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6.0,10.0,240.0,6/24/1968,Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6.0,9.0,235.0,4/7/1946,Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7.0,2.0,225.0,4/16/1947,"University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6.0,1.0,162.0,3/9/1969,Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6.0,6.0,223.0,11/3/1974,San Jose State University


In [23]:
df_clean_player_data = df_player_data.dropna(how='all')

In [18]:
df_player_data.columns

Index(['name', 'year_start', 'year_end', 'position', 'height_feet',
       'height_inches', 'weight', 'birth_date', 'college'],
      dtype='object')

In [24]:
#make player_data dataframe dropping name, birth_date columns
cols_2 = ['year_start', 'year_end', 'position', 'height_feet',
       'height_inches', 'weight', 'college']
df_player_data_2= df_clean_player_data[cols_2].copy()
df_player_data_2.head()

,year_start,year_end,position,height_feet,height_inches,weight,college
0,1991,1995,F-C,6.0,10.0,240.0,Duke University
1,1969,1978,C-F,6.0,9.0,235.0,Iowa State University
2,1970,1989,C,7.0,2.0,225.0,"University of California, Los Angeles"
3,1991,2001,G,6.0,1.0,162.0,Louisiana State University
4,1998,2003,F,6.0,6.0,223.0,San Jose State University


In [4]:
#make dataframe using name and birthday
cols = ["name", "birth_date"]
player_name_birthday_df = df_player_data[cols].copy()
player_name_birthday_df.head()

,name,birth_date
0,Alaa Abdelnaby,6/24/1968
1,Zaid Abdul-Aziz,4/7/1946
2,Kareem Abdul-Jabbar,4/16/1947
3,Mahmoud Abdul-Rauf,3/9/1969
4,Tariq Abdul-Wahad,11/3/1974


In [5]:
df_season_stats.columns

Index(['Unnamed: 0', 'Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2',
       'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [6]:
cols_1 = ['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
        'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
df_season_stats_2 = df_season_stats[cols_1].copy()
df_season_stats_3= df_season_stats_2.dropna(how ="all")
df_season_stats_3.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,144.0,516.0,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,102.0,274.0,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,174.0,499.0,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,22.0,86.0,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,21.0,82.0,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [7]:
df_season_stats_4= df_season_stats_3.rename(columns={"Pos": "Position",
                                                    "FG%": "FG_percent",
                                                    "3P%":"3P_percent",
                                                    "2P%": "2P_percent",
                                                    "eFG%": "eFG_percent",
                                                    "FT%": "FT_percent",
                                                    "Tm": "Team"})
df_season_stats_4.count()

Year           24624
Player         24624
Position       24624
Age            24616
Team           24624
G              24624
GS             18233
MP             24138
FG             24624
FGA            24624
FG_percent     24525
3P             18927
3PA            18927
3P_percent     15416
2P             24624
2PA            24624
2P_percent     24496
eFG_percent    24525
FT             24624
FTA            24624
FT_percent     23766
ORB            20797
DRB            20797
TRB            24312
AST            24624
STL            20797
BLK            20797
TOV            19645
PF             24624
PTS            24624
dtype: int64

In [8]:
#Create Tables: Height-W, Height-Inches, Main Position, Secondary Position, Teams
height = [4,5,6,7]
df_height = pd.DataFrame(height, columns =["height_feet"])
df_height.head()

inches = [0,1,2,3,4,5,6,7,8,9,10,11]
df_inches = pd.DataFrame(inches, columns =["height_inches"])
df_inches.head()

position = df_season_stats.Pos.unique()
Pos_df = pd.DataFrame(position, columns =["position"]).dropna()
Pos_df



teams = df_season_stats.Tm.unique()
team_df = pd.DataFrame(teams, columns = ["team"]).dropna()
Pos_df

,position
0,G-F
1,SG
2,SF
3,F
4,G
5,F-C
6,PG
7,F-G
8,C
9,PF


In [9]:
colleges = df_player_data["college"].unique
colleges

<bound method Series.unique of 0                                         Duke University
1                                   Iowa State University
2                   University of California, Los Angeles
3                              Louisiana State University
4                               San Jose State University
5                                University of California
6                                      Indiana University
7                             Western Kentucky University
8                          Salem International University
9                                                     NaN
10                                  Pepperdine University
11                                 Long Island University
12                                Oral Roberts University
13                                      Hillsdale College
14                                      Baylor University
15                                 University of Oklahoma
16                                Northwe

In [10]:
colleges = df_player_data.college.unique()
colleges_2 = pd.DataFrame(colleges)
colleges_3 = colleges_2.rename(columns={0: "college"})
import numpy as np
#Change NaN College Name to N/A so can be added to Table
colleges_4= colleges_3.replace(np.nan, 'N/A', regex=True)
colleges_4



,college
0,Duke University
1,Iowa State University
2,"University of California, Los Angeles"
3,Louisiana State University
4,San Jose State University
5,University of California
6,Indiana University
7,Western Kentucky University
8,Salem International University
9,N/A


In [11]:
from sqlalchemy import create_engine
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [12]:
#connect to the specific database containing the NBA data tables
connection_string = "postgres:postgres@localhost:5432/NBA Data"
engine = create_engine(f'postgresql://{connection_string}')

In [13]:
#Check Table Name
engine.table_names()

['Players', 'Player_data', 'Colleges', 'Teams', 'SeasonStats', 'Positions']

In [14]:
#Import College Table
colleges_4.to_sql(name="Colleges", con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_Colleges"
DETAIL:  Key (college)=(Duke University) already exists.

[SQL: INSERT INTO "Colleges" (college) VALUES (%(college)s)]
[parameters: ({'college': 'Duke University'}, {'college': 'Iowa State University'}, {'college': 'University of California, Los Angeles'}, {'college': 'Louisiana State University'}, {'college': 'San Jose State University'}, {'college': 'University of California'}, {'college': 'Indiana University'}, {'college': 'Western Kentucky University'}  ... displaying 10 of 474 total bound parameter sets ...  {'college': 'Friendship Junior College'}, {'college': 'Northeast Mississippi Community College'})]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [ ]:
#Import Team Table
team_df.to_sql(name="Teams", con=engine, if_exists='append', index=False)

In [ ]:
#import player positions
Pos_df.to_sql(name="Positions", con=engine, if_exists='append', index=False)

In [ ]:
#import player name/birthday table
player_name_birthday_df.to_sql(name="Players", con=engine, if_exists='append', index=False)

In [15]:
#import season stats table
df_season_stats_4.to_sql(name="SeasonStats", con=engine, if_exists='append', index=False)

In [26]:
#import player_data table
df_player_data_2.to_sql(name="Player_data", con=engine, if_exists='append', index=False)

In [27]:
#confirm data loaded to SQL 
pd.read_sql_query('select * from "Player_data"', con=engine)

,college,height_feet,height_inches,weight,position,year_start,year_end,id
0,Duke University,6.0,10.0,240.0,F-C,1991,1995,1
1,Iowa State University,6.0,9.0,235.0,C-F,1969,1978,2
2,"University of California, Los Angeles",7.0,2.0,225.0,C,1970,1989,3
3,Louisiana State University,6.0,1.0,162.0,G,1991,2001,4
4,San Jose State University,6.0,6.0,223.0,F,1998,2003,5
5,University of California,6.0,9.0,225.0,F,1997,2008,6
6,Indiana University,6.0,7.0,220.0,F,1977,1981,7
7,Western Kentucky University,6.0,3.0,180.0,G,1957,1957,8
8,Salem International University,6.0,3.0,195.0,F,1947,1948,9
9,None,6.0,6.0,190.0,G-F,2017,2018,10
